# Quantum Citation Walk Demo

This notebook demonstrates quantum walk-based citation graph traversal with fallback to classical methods.

In [ ]:
import sys
sys.path.append('../..')

import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

from quantum_integration.synthetic_data_generator import SyntheticDataGenerator
from quantum_integration.quantum_citation_walker import QuantumCitationWalker

## 1. Load Synthetic Data

In [ ]:
# Generate synthetic corpus
generator = SyntheticDataGenerator(seed=42)
corpus = generator.generate_synthetic_corpus(
    language="en",
    size=20,
    domain="quantum_computing",
    citation_density=0.15
)

print(f"Generated corpus with {corpus.size} documents")
print(f"Citation network: {len(corpus.citations)} citations")
print(f"Adjacency matrix shape: {corpus.adjacency_matrix.shape}")

## 2. Visualize Citation Network

In [ ]:
# Create NetworkX graph
G = nx.DiGraph()
G.add_nodes_from(range(corpus.size))
G.add_edges_from(corpus.citations)

# Visualize
plt.figure(figsize=(12, 8))
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=True, node_color='lightblue', 
        node_size=500, arrowsize=20, font_size=10)
plt.title("Citation Network")
plt.show()

## 3. Run Quantum Walk

In [ ]:
# Initialize quantum walker
walker = QuantumCitationWalker(backend="qiskit_aer", shots=1024)

# Run quantum walk
start_nodes = [0, 1, 2]  # Start from first 3 papers
quantum_result = walker.traverse(
    adjacency_matrix=corpus.adjacency_matrix,
    semantic_weights=corpus.semantic_weights,
    start_nodes=start_nodes,
    max_steps=5
)

print(f"\nQuantum Walk Results:")
print(f"Method: {quantum_result['method']}")
print(f"Number of paths: {len(quantum_result['paths'])}")
print(f"\nTop 5 paths with relevance scores:")
for i, (path, score) in enumerate(zip(quantum_result['paths'][:5], 
                                       quantum_result['relevance_scores'][:5])):
    print(f"  {i+1}. Path: {path}, Score: {score:.4f}")

## 4. Run Classical Walk for Comparison

In [ ]:
# Run classical walk
classical_result = walker._classical_traverse(
    adjacency_matrix=corpus.adjacency_matrix,
    semantic_weights=corpus.semantic_weights,
    start_nodes=start_nodes,
    max_steps=5
)

print(f"\nClassical Walk Results:")
print(f"Method: {classical_result['method']}")
print(f"Number of paths: {len(classical_result['paths'])}")
print(f"\nTop 5 paths with relevance scores:")
for i, (path, score) in enumerate(zip(classical_result['paths'][:5], 
                                       classical_result['relevance_scores'][:5])):
    print(f"  {i+1}. Path: {path}, Score: {score:.4f}")

## 5. Compare Results

In [ ]:
# Compare path diversity
quantum_unique = len(set(tuple(p) for p in quantum_result['paths']))
classical_unique = len(set(tuple(p) for p in classical_result['paths']))

print(f"\nPath Diversity Comparison:")
print(f"Quantum unique paths: {quantum_unique}")
print(f"Classical unique paths: {classical_unique}")

# Compare average scores
quantum_avg = np.mean(quantum_result['relevance_scores'])
classical_avg = np.mean(classical_result['relevance_scores'])

print(f"\nAverage Relevance Scores:")
print(f"Quantum: {quantum_avg:.4f}")
print(f"Classical: {classical_avg:.4f}")

# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].bar(['Quantum', 'Classical'], [quantum_unique, classical_unique])
axes[0].set_ylabel('Unique Paths')
axes[0].set_title('Path Diversity')

axes[1].bar(['Quantum', 'Classical'], [quantum_avg, classical_avg])
axes[1].set_ylabel('Average Score')
axes[1].set_title('Relevance Scores')

plt.tight_layout()
plt.show()

## 6. Entanglement Analysis (Quantum Only)

In [ ]:
if 'entanglement_measure' in quantum_result:
    print(f"\nQuantum Entanglement Measure: {quantum_result['entanglement_measure']:.4f}")
    print("Higher values indicate more quantum entanglement in the walk.")
else:
    print("\nEntanglement measure not available (classical fallback was used)")